In [ ]:
!pip install konlpy

In [1]:
# https://wikidocs.net/50739 와 <딥러닝 책봇>을 참고하여 만들었습니다.

In [14]:
import pandas as pd
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fe065165650>)

In [6]:
train_data = pd.read_table('ratings.txt')

In [7]:
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [8]:
#리뷰 개수
print(len(train_data))

200000


In [15]:
#null 값 존재 유무
print(train_data.isnull().values.any())

True


In [16]:
#null 값 존재하는 행 제거
train_data = train_data.dropna(how='any')
print(train_data.isnull().values.any())

False


In [17]:
print(len(train_data))

199992


In [18]:
#한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [19]:
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [20]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [21]:
okt = Okt()
tokenized_data=[]
for sentence in train_data['document']:
  temp_X = okt.morphs(sentence, stem=True)
  temp_X = [word for word in temp_X if not word in stopwords]
  tokenized_data.append(temp_X)

In [22]:
model = Word2Vec(sentences = tokenized_data, size=100, window=5, min_count=5, workers = 4, sg=0)

In [23]:
print(model.wv.most_similar("최민식"))

[('박중훈', 0.8408295512199402), ('안성기', 0.8403974771499634), ('설경구', 0.8345321416854858), ('이민호', 0.8312073945999146), ('최민수', 0.8262532949447632), ('김창완', 0.8167034983634949), ('채민서', 0.8155742883682251), ('전도연', 0.8115435838699341), ('이정재', 0.8107573986053467), ('윤제문', 0.810517430305481)]


In [29]:
print(model.wv.most_similar("행복"))

[('희망', 0.913443386554718), ('삶', 0.8827692270278931), ('자신', 0.8724240064620972), ('아픔', 0.8714065551757812), ('외로움', 0.8626793622970581), ('삶다', 0.8565982580184937), ('죽음', 0.8555512428283691), ('용기', 0.8545616269111633), ('자유', 0.8516080379486084), ('살아가다', 0.851474940776825)]


In [28]:
model.save('nvmc.model')